In [106]:
import pandas as pd
import seaborn as sns
import numpy as np
from pandas.plotting import autocorrelation_plot
from pandas import datetime
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.model_selection import *
from sklearn.linear_model import *
from sklearn import *
from statsmodels.tsa.arima_model import ARIMA
import math
import matplotlib.pyplot as plt
from matplotlib import style
plt.style.use('bmh')
color=sns.color_palette()

<ipython-input-106-a2a8e6f87f4c>:5: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  from pandas import datetime


In [107]:
def parser(x):
    return datetime.strptime(x, '%Y-%m')

In [108]:
style.use('ggplot')

In [109]:
print("please wait. importing...")
data = pd.read_csv("data.csv")
df=pd.read_csv('airtel.csv')
n=pd.read_csv('airtel.csv')

please wait. importing...


In [110]:
print("importing finished.")


importing finished.


In [111]:
data['Total_Price'] = data['Quantity']*data['UnitPrice']
data = data[data.Quantity > 0]
data = data[data.UnitPrice > 0]
data = data[data.iloc[:, :] != '']
x=df[['High','Open','Low','Volume']].values
y=df['Close'].values


In [112]:
data['date'] = data['InvoiceDate'].map(lambda x: str(x)[:7])
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=0)

In [113]:
    Quantity_date=data[['date', 'Quantity']].drop_duplicates()
    Quantity_date_count=Quantity_date.groupby(['date'])['Quantity'].aggregate('sum').reset_index().sort_values(by='date', ascending=0)


In [114]:
regressor = LinearRegression()
regressor.fit(x_train,y_train)

LinearRegression()

In [115]:
date=list(Quantity_date_count['date'])
quantity=list(Quantity_date_count['Quantity'])
date_quantity = pd.DataFrame({'dates': date, 'quantity':quantity})
y_pred=regressor.predict(x_test)
result=pd.DataFrame({'Actual':y_test.flatten(),'Predicted':y_pred.flatten()})
result.head(25)

,Actual,Predicted
0,293.806000,294.520262
1,389.691010,390.651577
2,337.686005,339.567961
3,146.214005,147.424404
4,303.260986,302.778833
5,297.615997,294.593135
6,306.199005,304.057382
7,14.297800,14.581266
8,305.281006,302.477483
9,269.112000,269.765795


In [116]:
date_quantity.index = date_quantity['dates'].map(lambda x: parser(x))
date_quantity['quantity'] = date_quantity['quantity'].map(lambda x: float(x))
graph=result.head(25)
graph.plot(kind='line')

ValueError: time data '31-10-2' does not match format '%Y-%m'

In [ ]:
date_quantity = date_quantity.fillna(date_quantity.bfill())
date_quantity = date_quantity['quantity'].resample('MS').mean()
df.shape



autocorrelation_plot(date_quantity)
plt.show()
plt.figure(figsize=(16,8))
plt.title('Airtel')
plt.xlabel('Time')
plt.ylabel('Close Price INR(₹)')
plt.plot(df['Close'])
plt.show()
quantity = date_quantity.values
df=df["Close"]

size = int(len(quantity) * 0.66)
train, test = quantity[0:size], quantity[size:len(quantity)]
history = [x for x in train]
predictions = list()
for t in range(len(test)):
    model = ARIMA(history, order=(2 ,2 ,0))
    model_fit = model.fit(disp=0)
    output = model_fit.forecast()
    yhat = output[0]
    predictions.append(yhat[0])
    obs = test[t]
    history.append(obs)
    print('predicted=%f, expected=%f' % (yhat, obs))

pred = np.array(predictions)

error = mean_squared_error(test, predictions)
print('Test MSE: %.3f' % error)


In [ ]:
plt.plot(test)
plt.xlabel('Axis')
plt.plot(predictions, color='green')
plt.show()


In [ ]:
future_days=25

n['Prediction']=n[["Close"]].shift(-future_days)
df=n.loc[:,('Close','Prediction')]

X=np.array(df.drop(['Prediction'],1))[:future_days]

y=np.array(df['Prediction'])[:future_days]
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.20)

x_future=df.drop(['Prediction'],1)[:-future_days]
x_future.tail(future_days)
x_future=np.array(x_future)
lr=LinearRegression().fit(x_train,y_train)
lr_prediction=lr.predict(x_future)


predictions=lr_prediction
valid=df[X.shape[0]:]
valid['Prediction']=predictions

plt.figure(figsize=(30,15))
plt.title('Airtel')
plt.xlabel('Time')
plt.ylabel('Close Price INR(₹)')
plt.plot(df['Close'])
plt.plot(valid[['Close','Prediction']])
plt.legend(['Orig','Val','Pred'])
plt.show()